In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import datetime

In [ ]:
dat=pd.read_csv('NASDAQOMX-NDX.csv')


In [ ]:
dat=dat.iloc[::-1]
dat.dropna(inplace=True)
dat = dat.reset_index(drop = True)

In [ ]:
date = dat[['Trade Date']]
dat.drop('Trade Date',inplace=True,axis=1)
dat=pd.concat([date,dat],axis=1)
dat['Trade Date'] = pd.to_datetime(dat['Trade Date'],format='%Y-%m-%d')


In [ ]:
dat

In [ ]:
plt.plot(dat['Trade Date'],dat["Index Value"])

In [ ]:
def typical(high,low,close,volume):
    mf = ((high + low + close)/3)*volume
    return mf     

In [ ]:
#EMA
def EMA(data):
    dats = data.astype(float)
    data['EMA'] = dats.ewm(span = 20).mean()
    return data['EMA']

In [ ]:
def MFI(data):
    data = pd.DataFrame(data)
    returns = data - data.shift(1)
    returns.dropna()
    up,down = returns.copy(),returns.copy()
    up[up<0] = 0
    down[down>0] = 0
    upm = up.mean()
    downm = down.abs().mean()
    
    mfi = 100 - (100/(1+(upm/downm)))
    return mfi

In [ ]:
#RSI
def RSI(data):
    data = pd.DataFrame(data)
    returns = data - data.shift(1)
    returns.dropna()
    up,down = returns.copy(),returns.copy()
    up[up<0] = 0
    down[down>0] = 0
    upm = up.mean()
    downm = down.abs().mean()
    rsi = 100 - (100/(1+(upm/downm)))
    return rsi 

In [ ]:
def MACD(ema):
    high = 0.075 * ema
    low = 0.15 * ema
    macd = high - low
    return macd

In [ ]:
def Oscillator(data):

    curr = data[-1]
    data = list(data)
    high = max(data)
    low = min(data)
    so = ((curr - low)/(high - low))*100 
    return so

In [ ]:
dat['MF'] = typical(dat['High'],dat['Low'],dat['Index Value'],dat['Total Market Value'])

In [ ]:
dat['EMA'] = EMA(dat.iloc[:,[3]])

In [ ]:
dat['RSI'] = dat['Index Value'].rolling(7).apply(RSI)

In [ ]:
dat['MFI'] = dat['MF'].rolling(7).apply(MFI)

In [ ]:
dat['SO'] = dat['Index Value'].rolling(7).apply(Oscillator)

In [ ]:
dat.index=date["Trade Date"]

In [ ]:
dat['MACD'] = MACD(dat.iloc[:,[7]])

In [ ]:
dat['MACD_Signal'] = MACD(dat.iloc[:,[11]])

In [ ]:
dat.drop(['High','Low','Total Market Value','Dividend Market Value','MF','MACD_Signal'],axis = 1,inplace = True)

In [ ]:
dat.drop("Trade Date",axis=1,inplace=True)

In [ ]:
xdata = dat.dropna()

In [ ]:
dates = xdata.copy()

In [ ]:
#xdata= dates.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xdata = pd.DataFrame(scaler.fit_transform(xdata))

In [ ]:
xdata = xdata.rename({0:'Index Value',1:'EMA',2:'RSI',3:'MFI',4:'SO',5:'MACD'},axis = 1)

In [ ]:
xdata.index = dates.index

In [ ]:
ydata = xdata['Index Value'].shift(-1)

In [ ]:
xdata = xdata.drop('Index Value',axis = 1)

In [ ]:
x_trains, y_trains = xdata[:3000],ydata[:3000]

In [ ]:
y_trains = pd.DataFrame(y_trains)

In [ ]:
x_tests, y_tests = xdata[3000:],ydata[3000:]

In [ ]:
y_tests = pd.DataFrame(y_tests)

In [ ]:
y_tests.dropna(inplace = True)

In [ ]:
x_tests  = x_tests[:-1]

In [ ]:
xdata  = xdata[:-1]

In [ ]:
ydata.dropna(inplace = True)

In [ ]:
from __future__ import division
import random
import math
import time

In [ ]:
"""Least Squares Support Vector Regression."""
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.gaussian_process import kernels
from scipy.sparse import linalg


class LSSVR(BaseEstimator, RegressorMixin):
    def __init__(self, C=None, kernel=None, gamma=10,polyconst=1,degree=2):
        self.supportVectors      = None
        self.supportVectorLabels = None
        self.C = C
        self.polyconst = float(1)
        self.gamma = gamma
        self.degree = degree
        self.kernel= kernel
        self.idxs  = None
        self.K = None
        self.bias = None 
        self.alphas = None
            

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, x_train, y_train):
        
        if type(self.idxs) == type(None):
            self.idxs=np.full(x_train.shape[0],True, dtype=bool)
            
             
        self.supportVectors      = x_train.loc[self.idxs, :]
        self.supportVectorLabels = y_train.loc[self.idxs]
               
        K = self.kernel_func(self.kernel, x_train, self.supportVectors, self.gamma)
        
        self.K = K
        OMEGA = K
        OMEGA[self.idxs, np.arange(OMEGA.shape[1])] =OMEGA[self.idxs, np.arange(OMEGA.shape[1])] + 1/self.C
        
        D = np.zeros(np.array(OMEGA.shape) + 1)
        D[1:,1:] = OMEGA
        D[0, 1:] += 1
        D[1:,0 ] += 1

        n = len(self.supportVectorLabels) + 1
        t = np.zeros(n)
        #t = pd.DataFrame(t)                      #Comment this line while running main function, nd run this block again uncommenting this line when running optimizer fn. 
        t[1:n] = self.supportVectorLabels.values
    
    
        try:
            z = linalg.lsmr(D.T, t)[0]
        except:
            z = np.linalg.pinv(D).T @ t.ravel()

        self.bias   = z[0]
        self.alphas = z[1:]
        self.alphas = self.alphas[self.idxs]

        return self

    def predict(self, x_test):
        K = self.kernel_func(self.kernel, x_test, self.supportVectors, self.gamma)

        return (K @ self.alphas) + self.bias
    
    def kernel_func(self, kernel, u, v, gamma):
        if kernel == 'linear':
            k = np.dot(u, v.T)
        if kernel == 'rbf':
            k = rbf_kernel(u, v, gamma=gamma)
            #k = np.exp(-1.0*self.gamma*np.dot(np.subtract(v,u).T,np.subtract(v,u))) #rbf expression
        if kernel == 'polynomial':
            k = (np.dot(u,v.T) + self.polyconst)**self.degree

            
        return k

In [ ]:
"""Least Squares Support Vector Regression."""
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.gaussian_process import kernels
from scipy.sparse import linalg


class LSSVR1(BaseEstimator, RegressorMixin):
    def __init__(self, C=None, kernel=None, gamma=10,polyconst=1,degree=2):
        self.supportVectors      = None
        self.supportVectorLabels = None
        self.C = C
        self.polyconst = float(1)
        self.gamma = gamma
        self.degree = degree
        self.kernel= kernel
        self.idxs  = None
        self.K = None
        self.bias = None 
        self.alphas = None
            

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, x_train, y_train):
        
        if type(self.idxs) == type(None):
            self.idxs=np.full(x_train.shape[0],True, dtype=bool)
            
             
        self.supportVectors      = x_train.loc[self.idxs, :]
        self.supportVectorLabels = y_train.loc[self.idxs]
               
        K = self.kernel_func(self.kernel, x_train, self.supportVectors, self.gamma)
        
        self.K = K
        OMEGA = K
        OMEGA[self.idxs, np.arange(OMEGA.shape[1])] =OMEGA[self.idxs, np.arange(OMEGA.shape[1])] + 1/self.C
        
        D = np.zeros(np.array(OMEGA.shape) + 1)
        D[1:,1:] = OMEGA
        D[0, 1:] += 1
        D[1:,0 ] += 1

        n = len(self.supportVectorLabels) + 1
        t = np.zeros(n)
        t = pd.DataFrame(t)                      #Comment this line while running main function, nd run this block again uncommenting this line when running optimizer fn. 
        t[1:n] = self.supportVectorLabels.values
    
    
        try:
            z = linalg.lsmr(D.T, t)[0]
        except:
            z = np.linalg.pinv(D).T @ t.ravel()

        self.bias   = z[0]
        self.alphas = z[1:]
        self.alphas = self.alphas[self.idxs]

        return self

    def predict(self, x_test):
        K = self.kernel_func(self.kernel, x_test, self.supportVectors, self.gamma)

        return (K @ self.alphas) + self.bias
    
    def kernel_func(self, kernel, u, v, gamma):
        if kernel == 'linear':
            k = np.dot(u, v.T)
        if kernel == 'rbf':
            k = rbf_kernel(u, v, gamma=gamma)
            #k = np.exp(-1.0*self.gamma*np.dot(np.subtract(v,u).T,np.subtract(v,u))) #rbf expression
        if kernel == 'polynomial':
            k = (np.dot(u,v.T) + self.polyconst)**self.degree

            
        return k

In [ ]:
#PSO
#--- MAIN ---------------------------------------------------------------------+

class Particle:
    def __init__(self,bounds):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(random.uniform(bounds[i][0],bounds[i][1]))

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i<self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i.copy()
            self.err_best_i=self.err_i
                    
    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant
        
        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()
            
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i]<bounds[i][0]:
                self.position_i[i]=bounds[i][0]
        
        
def PSO(costFunc, bounds, num_particles, maxiter, verbose=False):
    global num_dimensions

    num_dimensions=len(bounds)
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group

    # establish the swarm
    swarm=[]
    for i in range(0,num_particles):
        swarm.append(Particle(bounds))

    # begin optimization loop
    i=0
    while i<maxiter:
        if verbose: print(f'iter: {i:>4d}, best solution: {err_best_g:10.6f}')
            
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i<err_best_g or err_best_g==-1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)
        
        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1

    # print final results
    print('\nFINAL SOLUTION:')
    print(f'   > {pos_best_g}')
    print(f'   > {err_best_g}\n')

    pass

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
fold_count=10 

def load_csvdata():
    global X
    global Y
    
    X = xdata
    Y = ydata

def svrPso(params):
    kf = TimeSeriesSplit(n_splits=fold_count)
    for train, test in kf.split(X):
        Total = 0
         
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y.iloc[train], Y.iloc[test]
        nn = LSSVR(kernel = 'rbf',C=params[0], gamma = params[1])#Change Kernel name here(for poly kernel add 2 more parameters to optimize[degree,polyconst])
        nn.fit(X_train,y_train)
        result = nn.predict(X_test);
        thisError = calsError(y_test, result)
        Total = Total + thisError    
    ErrorCV = Total/fold_count; 
    print('Optimizing the Parameters ..... C = {c}, gamma={e}, MSE={m}'.format(c=params[0], e=params[1], m=ErrorCV))
    return ErrorCV

def calsError(y_test, result):
    y_pred = pd.DataFrame(result)
    y_pred.index = y_test.index
    data = pd.concat([y_test,y_pred],axis=1)
    data['Error'] = (data['Index Value'] - data[0])**2
    err = np.sqrt(np.sum(data['Error']))
    
    return err
    
def main_run():
    
        load_csvdata()
    

        #For PSO
        bounds=[(0.01,1000),(0.0002,100)] 
        PSO(svrPso , bounds ,num_particles=5 ,maxiter=50)
        
        
        print(" ")
        print("************ Objective Function optimized *****************")
        print(" ")
        print(" ")

In [ ]:
print("************  Initializing Optimization *****************")
tic = time.time()
main_run()
toc = time.time()
print("************  Optimization Finished *****************")
print('Time Taken is {time} secs'.format(time = (toc - tic)))

In [ ]:
model = LSSVR1(kernel='rbf',C =list(pos_best_g[0]),gamma = list(pos_best_g[1]))
model.fit(x_trains,y_trains)

In [ ]:
res = model.predict(x_tests)
res = pd.DataFrame(res)
res.index = y_tests.index
res.rename({0:'Index value'},axis =1,inplace=True)


In [ ]:
plt.figure(figsize = (20,14))
plt.plot(y_tests,color="#0A0AFF",alpha=0.5,label="Test Data")
plt.plot(res,color="#AF0000",label="Predections")
plt.title("Comparision of Test data and Predictions Linear Kernel",fontsize=15)
#blue_patch=mpatches.Patch(color="#0A0AFF",label="Test Data")
#red_patch=mpatches.Patch(color="#AF0000",label="Predictions")
plt.legend(fontsize="large")